# Generating dataset 


In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as T
from torchvision import datasets
import copy
import os
from tqdm.notebook import tqdm
import numpy as np 
np.set_printoptions(suppress=True)

import random

# Create mini-mnist C dataset for mini testing
get 1000 samples from each corruption dataset * 15 corruptions = 15000 sample images and labels

In [2]:
random.seed(10)
randomlist = random.sample(range(0, 10000), 1000)
# randomlist = [i for i in range(1000)]
# print(randomlist)
CORRUPTION_TYPES = ['identity', 
         'shot_noise', 'impulse_noise','glass_blur','motion_blur',
         'shear', 'scale',  'rotate',  'brightness',  'translate',
         'stripe', 'fog','spatter','dotted_line', 'zigzag',
         'canny_edges']

path_root = '../data/MNIST_C'
# clist = os.listdir(path_root)
ilist = []
llist = []
for corruption in CORRUPTION_TYPES:
    path_images = os.path.join(path_root, corruption, 'test_images.npy')
    path_labels = os.path.join(path_root, corruption, 'test_labels.npy')
    images = np.load(path_images)
    labels = np.load(path_labels)
    
    ilist.append(images[randomlist])
    llist.append(labels[randomlist])

selected_images = np.vstack(ilist)
selected_labels = np.hstack(llist)
print(selected_images.shape) #(16000, 28, 28, 1)
print(selected_labels.shape) #(16000,)

(16000, 28, 28, 1)
(16000,)


In [3]:
#tranform to tensorized images (int8->float32,0-1) & one-hot vector for labels
transform_tohot = T.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
selected_images_tensorized = torch.stack([T.ToTensor()(im) for im in selected_images])
selected_labels_tensorized = torch.stack([transform_tohot(label) for label in selected_labels])
print(selected_images_tensorized.shape) #torch.Size([16000, 1, 28, 28])
print(selected_labels_tensorized.shape) #torch.Size([16000, 10])

torch.Size([16000, 1, 28, 28])
torch.Size([16000, 10])


In [4]:
# save tensorized dataset
path_save = os.path.join(path_root, 'mnistc_mini.pt')
torch.save((selected_images_tensorized, selected_labels_tensorized), path_save)
print('dataset saved to disk')

dataset saved to disk


In [31]:
##################################
# shuffle the order original mnistc (for visualization)
############################
# # print(randomlist)
# CORRUPTION_TYPES = ['identity', 
#          'shot_noise', 'impulse_noise','glass_blur','motion_blur',
#          'shear', 'scale',  'rotate',  'brightness',  'translate',
#          'stripe', 'fog','spatter','dotted_line', 'zigzag',
#          'canny_edges']
# CORRUPTION_TYPES = ['impulse_noise']

# path_root = '../data/MNIST_C'
# # clist = os.listdir(path_root)
# for corruption in CORRUPTION_TYPES:
#     randomlist = random.sample(range(0, 10000), 10000)

#     path_images = os.path.join(path_root, corruption, 'test_images.npy')
#     path_labels = os.path.join(path_root, corruption, 'test_labels.npy')
#     path_images_shuffled = os.path.join(path_root, corruption, 'test_images_shuffled.npy')
#     path_labels_shuffled = os.path.join(path_root, corruption, 'test_labels_shuffled.npy')
    
#     images = np.load(path_images)[randomlist]
#     labels = np.load(path_labels)[randomlist]
    
#     np.save(path_images_shuffled, images)
#     np.save(path_labels_shuffled, labels)

# print('shuffled images and labels are saved')

shuffled images and labels are saved
